<a href="https://colab.research.google.com/github/brianpenrod/project_alpha_venezuela_risk/blob/main/PROJECT_ALPHA_%E2%80%94_BUILD_ALL_FIGURES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip -q install kaleido


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 1.6 MB/s eta 0:00:00


In [3]:
# ==========================================
# PROJECT ALPHA — BUILD ALL FIGURES (ONE SCRIPT)
# Figures:
#   Fig 1: Debt → Control Waterfall (Plotly)
#   Fig 2: Kinetic Latency (Mach 5–10 band) (Plotly)
#   Fig 3: Orthogonality Matrix Heatmap (Plotly)
# Outputs (optional): PNGs to ./figures + sensitivity table CSV
# ==========================================

import os
import numpy as np
import pandas as pd
import plotly.graph_objects as go

# ----------------------------
# GLOBAL CONFIG
# ----------------------------
TEMPLATE = "plotly_dark"
FONT_FAMILY = "Arial"
OUTPUT_DIR = "figures"
EXPORT_PNG = True  # set False if you only want interactive figures in Colab

os.makedirs(OUTPUT_DIR, exist_ok=True)

def try_export_png(fig: go.Figure, path: str) -> None:
    """Export PNG if kaleido is installed; otherwise skip gracefully."""
    if not EXPORT_PNG:
        return
    try:
        fig.write_image(path, scale=2)  # requires kaleido
        print(f"Saved: {path}")
    except Exception as e:
        print(f"[Skipped PNG export] {path}\nReason: {e}\nTip: pip install kaleido")

# ----------------------------
# HELPERS
# ----------------------------
def control_multiplier_from_recovery(rr_pct: float) -> float:
    """
    Your heuristic:
      m(rr) = 1 + (1 - rr)
    where rr is in [0,1]. If rr_pct is in percent:
      m = 2 - rr_pct/100
    Example: rr=20% => m=1.8x
    """
    return 2.0 - (rr_pct / 100.0)

def minutes(distance_km: float, speed_kmh: float) -> float:
    return (distance_km / speed_kmh) * 60.0

# ==========================================
# INPUTS — FINANCIAL FORENSICS (Waterfall)
# ==========================================
total_credit_b = 65.0
repaid_b = 51.0
outstanding_b = total_credit_b - repaid_b

base_rr = 20.0  # percent
mult = control_multiplier_from_recovery(base_rr)
implied_strategic_value_b = outstanding_b * mult

# Optional: sensitivity table (nice for appendix / sanity)
rr_grid = [10, 20, 30, 40]
sens = []
for rr in rr_grid:
    m = control_multiplier_from_recovery(rr)
    sens.append({
        "recovery_rate_pct": rr,
        "control_multiplier_x": round(m, 2),
        "implied_strategic_value_b": round(outstanding_b * m, 1),
        "expected_cash_recovery_b": round(outstanding_b * (rr / 100.0), 1),
    })
df_sensitivity = pd.DataFrame(sens)
print("\nSensitivity (Recovery → Multiplier):")
display(df_sensitivity)
df_sensitivity.to_csv(os.path.join(OUTPUT_DIR, "sensitivity_recovery_multiplier.csv"), index=False)

# ==========================================
# FIGURE 1 — WATERFALL: Debt → Control Value
# ==========================================
fig1 = go.Figure(go.Waterfall(
    orientation="v",
    measure=["relative", "relative", "total", "total"],
    x=["Total Credit", "Principal Repaid", "Outstanding Debt", "Implied Strategic Value"],
    y=[total_credit_b, -repaid_b, outstanding_b, implied_strategic_value_b],
    text=[
        f"${total_credit_b:.1f}B",
        f"-${repaid_b:.1f}B",
        f"${outstanding_b:.1f}B",
        f"${implied_strategic_value_b:.1f}B (m={mult:.2f}x, rr={base_rr:.0f}%)"
    ],
    textposition="outside",
    connector={"line": {"color": "#7f8c8d"}},
    # NOTE: In Plotly Waterfall:
    #   - "decreasing" applies to negative values (e.g., repaid)
    #   - "increasing" applies to positive values
    decreasing={"marker": {"color": "#00cc96"}},  # green for -repaid (money in / reduction)
    increasing={"marker": {"color": "#ef553b"}},  # red for positive steps
    totals={"marker": {"color": "#ffa15a"}}       # orange for totals
))

fig1.update_layout(
    title=dict(text=f"<b>FORENSIC SCENARIO: Debt → Control Value (rr={base_rr:.0f}%)</b>", font=dict(size=20)),
    template=TEMPLATE,
    font=dict(family=FONT_FAMILY),
    showlegend=False,
    height=550,
    paper_bgcolor="rgba(0,0,0,0)",
    plot_bgcolor="rgba(0,0,0,0)",
)
fig1.show()
try_export_png(fig1, os.path.join(OUTPUT_DIR, "figure_1_debt_to_control_waterfall.png"))

# ==========================================
# INPUTS — KINETIC LATENCY (Mach band)
# ==========================================
targets = ["Puerto Rico (Air Base)", "Panama Canal (Logistics)", "Miami (Command Node)", "Houston (Energy Node)"]
dist_km = [900, 1100, 2200, 3300]
df_nodes = pd.DataFrame({"Target": targets, "Distance_km": dist_km})

# Speed of sound ~ 1235 km/h is a simplification (varies w/ altitude/temp).
speed_of_sound_kmh = 1235.0

mach_min, mach_max = 5.0, 10.0
mach_point = 7.5  # point estimate plotted (you used this earlier)

v_min = mach_min * speed_of_sound_kmh
v_max = mach_max * speed_of_sound_kmh
v_point = mach_point * speed_of_sound_kmh

df_nodes["t_point_min"] = df_nodes["Distance_km"].apply(lambda d: minutes(d, v_point))
df_nodes["t_fast_min"]  = df_nodes["Distance_km"].apply(lambda d: minutes(d, v_max))
df_nodes["t_slow_min"]  = df_nodes["Distance_km"].apply(lambda d: minutes(d, v_min))

# Error bars: show Mach 5–10 band around the Mach 7.5 point
err_plus  = df_nodes["t_slow_min"] - df_nodes["t_point_min"]  # upward to slower time
err_minus = df_nodes["t_point_min"] - df_nodes["t_fast_min"]  # downward to faster time

# ==========================================
# FIGURE 2 — KINETIC LATENCY SCATTER (Mach band)
# ==========================================
fig2 = go.Figure()

fig2.add_trace(go.Scatter(
    x=df_nodes["Distance_km"],
    y=df_nodes["t_point_min"],
    mode="markers+text",
    text=df_nodes["Target"],
    textposition="top center",
    marker=dict(size=14, line=dict(width=2, color="white")),
    error_y=dict(
        type="data",
        symmetric=False,
        array=err_plus,
        arrayminus=err_minus
    ),
    name=f"Targets (Mach {mach_min:.0f}–{mach_max:.0f} band)"
))

# Threshold lines
fig2.add_hline(
    y=15, line_dash="dot", line_color="#ef553b", line_width=2,
    annotation_text="<b>DECISION THRESHOLD (15m)</b>",
    annotation_position="bottom right",
    annotation_font=dict(color="#ef553b", size=12),
)
fig2.add_hline(
    y=30, line_dash="dot", line_color="#ffa15a", line_width=2,
    annotation_text="<b>PLANNING THRESHOLD (30m)</b>",
    annotation_position="top right",
    annotation_font=dict(color="#ffa15a", size=12),
)

fig2.update_layout(
    title=f"<b>KINETIC LATENCY: Reaction Windows (Mach {mach_min:.0f}–{mach_max:.0f}, point={mach_point:.1f})</b>",
    xaxis_title="Distance (km)",
    yaxis_title="Flight Time (Minutes)",
    template=TEMPLATE,
    font=dict(family=FONT_FAMILY),
    height=550,
    showlegend=False,
    paper_bgcolor="rgba(0,0,0,0)",
    plot_bgcolor="rgba(0,0,0,0)",
)
fig2.show()
try_export_png(fig2, os.path.join(OUTPUT_DIR, "figure_2_kinetic_latency_mach_band.png"))

# ==========================================
# INPUTS — ORTHOGONALITY MATRIX (Risk heatmap)
# ==========================================
risk_data = pd.DataFrame({
    "Domain": ["Financial", "Infrastructure", "Digital", "Kinetic"],
    "Mechanism": ["Cross-Default Clauses", "Port Control", "Data Wipe (Kill Switch)", "Forward Base"],
    "Risk_Level_Text": ["HIGH", "MED", "CRITICAL", "EXISTENTIAL"]
})

severity_map = {
    "MED": 0.5,
    "HIGH": 0.75,
    "CRITICAL": 0.9,
    "EXISTENTIAL": 1.0
}
risk_data["Severity_Score"] = risk_data["Risk_Level_Text"].map(severity_map)

# Build a true "orthogonality" matrix:
# - rows = mechanisms
# - cols = domains
# - fill zeros where domain != mechanism's mapped domain
domains = ["Financial", "Infrastructure", "Digital", "Kinetic"]
mechanisms = risk_data["Mechanism"].tolist()

mat = pd.DataFrame(0.0, index=mechanisms, columns=domains)
for _, row in risk_data.iterrows():
    mat.loc[row["Mechanism"], row["Domain"]] = float(row["Severity_Score"])

# Heatmap text: show score where >0 else blank
text = mat.applymap(lambda v: f"{v:.1f}" if v > 0 else "").values

# ==========================================
# FIGURE 3 — ORTHOGONALITY MATRIX HEATMAP (Plotly)
# ==========================================
fig3 = go.Figure(data=go.Heatmap(
    z=mat.values,
    x=mat.columns.tolist(),
    y=mat.index.tolist(),
    colorscale="Reds",
    zmin=0, zmax=1,
    text=text,
    texttemplate="%{text}",
    textfont=dict(color="white", size=14),
    colorbar=dict(title="Strategic Severity Index (0.0–1.0)"),
    hovertemplate="Domain: %{x}<br>Mechanism: %{y}<br>Severity: %{z:.2f}<extra></extra>",
))

fig3.update_layout(
    title="<b>ORTHOGONALITY MATRIX: Risk Factor Isolation & Severity</b>",
    xaxis_title="<b>Risk Domain</b>",
    yaxis_title="<b>Trigger Mechanism</b>",
    template=TEMPLATE,
    font=dict(family=FONT_FAMILY),
    height=550,
    paper_bgcolor="rgba(0,0,0,0)",
    plot_bgcolor="rgba(0,0,0,0)",
)
fig3.show()
try_export_png(fig3, os.path.join(OUTPUT_DIR, "figure_3_orthogonality_matrix.png"))

print("\nDone. Figures (and sensitivity CSV) are in ./figures/")
# ==========================================
# FIGURE 4 — STRUCTURAL BREAK (Regime Change / Non-Stationarity)
# Expected columns:
#   date, oil_prod_bpd, debt_service_usd, repayment_usd
# ==========================================

import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

TIME_SERIES_CSV = "data/venezuela_regime_timeseries.csv"
BREAK_DATE = "2024-10-01"  # adjust if needed

# --- Load ---
if not os.path.exists(TIME_SERIES_CSV):
    raise FileNotFoundError(
        f"Can't find {TIME_SERIES_CSV}. "
        "In Colab, either upload it, or create the repo folder structure with data/..."
    )

df = pd.read_csv(TIME_SERIES_CSV)

required = {"date", "oil_prod_bpd", "debt_service_usd", "repayment_usd"}
missing = required - set(df.columns)
if missing:
    raise ValueError(f"Missing required columns: {sorted(missing)}")

df["date"] = pd.to_datetime(df["date"], errors="coerce")
df = df.dropna(subset=["date"]).sort_values("date")

for col in ["oil_prod_bpd", "debt_service_usd", "repayment_usd"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")

break_dt = pd.to_datetime(BREAK_DATE)

# --- Transform for readable units ---
df["oil_prod_mbpd"] = df["oil_prod_bpd"] / 1e6          # million barrels/day
df["debt_service_bil"] = df["debt_service_usd"] / 1e9   # $ billions
df["repayment_bil"] = df["repayment_usd"] / 1e9         # $ billions

pre = df[df["date"] < break_dt]
post = df[df["date"] >= break_dt]

def pct_change(pre_val, post_val):
    if pd.isna(pre_val) or pd.isna(post_val) or pre_val == 0:
        return None
    return 100 * (post_val - pre_val) / pre_val

oil_pre, oil_post = pre["oil_prod_mbpd"].mean(), post["oil_prod_mbpd"].mean()
debt_pre, debt_post = pre["debt_service_bil"].mean(), post["debt_service_bil"].mean()
rep_pre, rep_post = pre["repayment_bil"].mean(), post["repayment_bil"].mean()

oil_pc = pct_change(oil_pre, oil_post)
debt_pc = pct_change(debt_pre, debt_post)
rep_pc = pct_change(rep_pre, rep_post)

# --- Plot ---
fig, ax1 = plt.subplots(figsize=(11, 6))
ax2 = ax1.twinx()

l1, = ax1.plot(df["date"], df["oil_prod_mbpd"], linewidth=2, label="Oil production (Mbpd)")
l2, = ax2.plot(df["date"], df["debt_service_bil"], linewidth=2, label="Debt service ($B)")
l3, = ax2.plot(df["date"], df["repayment_bil"], linewidth=2, label="Repayment ($B)")

# Break marker + regime shading
ax1.axvline(break_dt, linewidth=2, linestyle="--")
ax1.axvspan(break_dt, df["date"].max(), alpha=0.12)

# Labels / title
ax1.set_title("Figure 4 — Structural Break (Regime Change / Non-Stationarity)")
ax1.set_xlabel("Date")
ax1.set_ylabel("Oil production (Mbpd)")
ax2.set_ylabel("USD ($B)")

# Date formatting
ax1.xaxis.set_major_locator(mdates.AutoDateLocator())
ax1.xaxis.set_major_formatter(mdates.ConciseDateFormatter(ax1.xaxis.get_major_locator()))
ax1.grid(True, axis="both", alpha=0.25)

# Combined legend
lines = [l1, l2, l3]
labels = [ln.get_label() for ln in lines]
ax1.legend(lines, labels, loc="upper left", frameon=True)

# Quick annotation box (optional, but helpful in a report)
def fmt_pc(x):
    return "n/a" if x is None else f"{x:+.1f}%"

note = (
    f"Break date: {BREAK_DATE}\n"
    f"Post vs Pre mean change:\n"
    f"• Oil: {fmt_pc(oil_pc)}\n"
    f"• Debt service: {fmt_pc(debt_pc)}\n"
    f"• Repayment: {fmt_pc(rep_pc)}"
)
ax1.text(
    0.02, 0.02, note,
    transform=ax1.transAxes,
    va="bottom", ha="left",
    fontsize=9,
    bbox=dict(boxstyle="round,pad=0.35", alpha=0.15)
)

plt.tight_layout()

# --- Save for GitHub ---
os.makedirs("figures", exist_ok=True)
out_path = "figures/fig4_structural_break.png"
plt.savefig(out_path, dpi=200, bbox_inches="tight")
plt.show()

print(f"Saved: {out_path}")

# ==========================================
# FIGURE 5 — MONTE CARLO (Scenario Valuation Band)
# IMPORTANT: This is not “truth.” It’s a distribution implied by assumptions.
# ==========================================

np.random.seed(42)
SIMS = 10_000

# Use the same outstanding face value as your Figure 1 logic (keep the story consistent)
BASE_DEBT_B = outstanding_b  # e.g., $14B from your waterfall

# Priors (assumptions)
# Strategic premium: centered ~1.5x; clipped to avoid nonsensical negatives
strategic_premium = np.random.normal(loc=1.5, scale=0.2, size=SIMS)
strategic_premium = np.clip(strategic_premium, 0.5, None)

# Oil recovery / asset upside factor
oil_recovery_factor = np.random.triangular(left=0.8, mode=1.0, right=1.5, size=SIMS)

implied_value_b = BASE_DEBT_B * strategic_premium * oil_recovery_factor

p05 = float(np.percentile(implied_value_b, 5))
p50 = float(np.percentile(implied_value_b, 50))
p95 = float(np.percentile(implied_value_b, 95))
mean_val = float(np.mean(implied_value_b))

fig5 = go.Figure()
fig5.add_trace(go.Histogram(
    x=implied_value_b,
    nbinsx=60,
    name="Implied Value Distribution"
))

# Reference lines
fig5.add_vline(x=p05, line_dash="dash", line_width=2, line_color="#ef553b",
               annotation_text=f"<b>P05 = ${p05:.1f}B</b>", annotation_position="top left")
fig5.add_vline(x=mean_val, line_dash="solid", line_width=2, line_color="#00cc96",
               annotation_text=f"<b>Mean = ${mean_val:.1f}B</b>", annotation_position="top right")
fig5.add_vline(x=p95, line_dash="dash", line_width=2, line_color="#ffa15a",
               annotation_text=f"<b>P95 = ${p95:.1f}B</b>", annotation_position="bottom right")

fig5.update_layout(
    title="<b>MONTE CARLO (Scenario): Implied Strategic/Control Value Distribution (n=10,000)</b>",
    xaxis_title="Implied Value (Billions USD)",
    yaxis_title="Frequency",
    template=TEMPLATE,
    font=dict(family=FONT_FAMILY),
    height=550,
    paper_bgcolor="rgba(0,0,0,0)",
    plot_bgcolor="rgba(0,0,0,0)",
    showlegend=False,
)

fig5.show()
try_export_png(fig5, os.path.join(OUTPUT_DIR, "figure_5_monte_carlo_valuation.png"))

print("\nFig 4/5 complete (if inputs existed).")



Sensitivity (Recovery → Multiplier):


,recovery_rate_pct,control_multiplier_x,implied_strategic_value_b,expected_cash_recovery_b
0,10,1.9,26.6,1.4
1,20,1.8,25.2,2.8
2,30,1.7,23.8,4.2
3,40,1.6,22.4,5.6


[Skipped PNG export] figures/figure_1_debt_to_control_waterfall.png
Reason: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido

Tip: pip install kaleido


[Skipped PNG export] figures/figure_2_kinetic_latency_mach_band.png
Reason: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido

Tip: pip install kaleido


/tmp/ipython-input-2125723375.py:217: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.



[Skipped PNG export] figures/figure_3_orthogonality_matrix.png
Reason: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido

Tip: pip install kaleido

Done. Figures (and sensitivity CSV) are in ./figures/


FileNotFoundError: Can't find data/venezuela_regime_timeseries.csv. In Colab, either upload it, or create the repo folder structure with data/...